<a href="https://colab.research.google.com/github/klisman02/Data-Analysis-Project/blob/main/reconhecimento_facial_friends_characters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================
# Instalação das dependências
# ==============================
!pip install --quiet tensorflow opencv-python keras-facenet scikit-learn matplotlib numpy


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.4 MB/s eta 0:00:00


In [ ]:
# ==============================
# Imports principais
# ==============================
import tensorflow as tf
import cv2
import numpy as np
from keras_facenet import FaceNet
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import os

print("TensorFlow:", tf.__version__)
print("OpenCV:", cv2.__version__)
print("NumPy:", np.__version__)


In [ ]:
# ==============================
# Baixar modelo de detecção de faces do OpenCV (SSD)
# ==============================
proto = "deploy.prototxt"
model = "res10_300x300_ssd_iter_140000.caffemodel"

!wget -q https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt -O $proto
!wget -q https://raw.githubusercontent.com/opencv/opencv_3rdparty/master/res10_300x300_ssd_iter_140000.caffemodel -O $model

# Carregar detector
face_net = cv2.dnn.readNetFromCaffe(proto, model)

# Inicializar FaceNet para embeddings
embedder = FaceNet()
l2_normalizer = Normalizer('l2')

print("✅ Modelos carregados com sucesso!")


In [ ]:
# ==============================
# Funções auxiliares
# ==============================
def detect_faces(img_bgr, conf_threshold=0.9):
    h, w = img_bgr.shape[:2]
    blob = cv2.dnn.blobFromImage(img_bgr, 1.0, (300, 300), (104, 177, 123))
    face_net.setInput(blob)
    detections = face_net.forward()

    faces = []
    boxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x1, y1, x2, y2) = box.astype("int")
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w-1, x2), min(h-1, y2)
            face = img_bgr[y1:y2, x1:x2]
            if face.size > 0:
                faces.append(face)
                boxes.append((x1, y1, x2, y2))
    return faces, boxes

def embed_faces(faces):
    embeddings = []
    for f in faces:
        f_rgb = cv2.cvtColor(f, cv2.COLOR_BGR2RGB)
        embs = embedder.embeddings([f_rgb])
        embeddings.append(l2_normalizer.transform(embs)[0])
    return np.array(embeddings)


In [ ]:
# !ls "/content/drive/MyDrive/"
# !ls "/content/drive/MyDrive/reconhecimento_facial/"
# !ls "/content/drive/MyDrive/reconhecimento_facial/train"
# !ls "/content/drive/MyDrive/reconhecimento_facial/val"

In [ ]:
# ==============================
# Carregar dataset (train e val)
# Estrutura esperada:
# /content/train/Chandler/*.jpg
# /content/train/Joey/*.jpg
# ...
# ==============================
def load_dataset(root_dir):
    X, y = [], []
    classes = sorted(os.listdir(root_dir))
    print("Classes:", classes)
    for label in classes:
        folder = os.path.join(root_dir, label)
        for fname in os.listdir(folder):
            path = os.path.join(folder, fname)
            img = cv2.imread(path)
            if img is None:
                continue
            faces, _ = detect_faces(img)
            if len(faces) > 0:
                embs = embed_faces([faces[0]])  # assume 1 rosto por imagem
                X.append(embs[0])
                y.append(label)
    return np.array(X), np.array(y)

X_train, y_train = load_dataset("/content/drive/MyDrive/reconhecimento_facial/train")
X_val, y_val = load_dataset("/content/drive/MyDrive/reconhecimento_facial/val")


In [ ]:
# ==============================
# Treinar classificador (SVM)
# ==============================
clf = SVC(kernel='linear', probability=True)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))


In [ ]:
# ==============================
# Testar em imagem com todos juntos
# ==============================
img = cv2.imread("/content/drive/MyDrive/reconhecimento_facial/friends-cast.webp")  # coloque uma foto do elenco todo
faces, boxes = detect_faces(img)

if faces:
    embs = embed_faces(faces)
    preds = clf.predict(embs)
    for (x1, y1, x2, y2), label in zip(boxes, preds):
        cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()
